In [ ]:
import requests, re
import pandas as pd, numpy as np
from bs4 import BeautifulSoup

In [ ]:
page = requests.get('http://www.hindilyrics.net/lyrics/lyricists.html')
soup = BeautifulSoup(page.content,'html.parser')
lyricists = soup.find_all('li')

In [ ]:
len(lyricists)

In [ ]:
THRESHOLD_NUM_SONGS = 400

In [ ]:
required_url=[]
for lyricist in lyricists:
    url = 'http://www.hindilyrics.net'+lyricist.find('a')['href'].replace('%20',' ')
    songs = requests.get(url)
    songs = BeautifulSoup(songs.content,'html.parser')
    content = re.findall(r'total\s+(\d+)\s+',str(songs))
    # Fetch two lyricists who have penned > THRESHOLD_NUM_SONGS     
    if len(content) > 0 and int(content[0])>THRESHOLD_NUM_SONGS:
        required_url.append(url)
    if len(required_url)==2:
        break

In [ ]:
required_url

In [ ]:
songs_list={}

for url in required_url:
    all_songs_url = [url]
    
    # Find all the songs list in the first page
    songs = requests.get(url)
    lyrics_list = BeautifulSoup(songs.content,'html.parser').find_all('li')
    
    # Find if additional pages are there
    more_pages = lyrics_list[-1].get_text().find('More Pages')
    if more_pages != -1:
        page_urls = lyrics_list[-1].find_all('a')[1:]
    
    # Collect the url of all the pages
    for songs_url in page_urls:
        all_songs_url.append('http://www.hindilyrics.net/lyrics/'+songs_url['href'])
    lyricist = re.findall(r'lyricist-([\w\s.]+)\.html',url)[0]
    
    songs_list[lyricist] = []
    i = 0
    for song_pages in all_songs_url:
        # Fetch all lyrics  page wise
        songs = requests.get(song_pages)
        lyrics_list = BeautifulSoup(songs.content,'html.parser').find_all('li')
        # Lyrics in a single page
        for lyrics in lyrics_list:
            try:
                url_pages = 'http://www.hindilyrics.net' + lyrics.find('a')['href'].replace('%20',' ')
                content = requests.get(url_pages)
                content = BeautifulSoup(content.content,'html.parser')
                text = re.sub(r'\n|\r','',content.find('pre').get_text())
                songs_list[lyricist].append(text)
            except:
                # Problem songs
                print('error in fetching:',url_pages)

In [ ]:
# Save & verify
for lyricist,songs in songs_list.items():
    with open(lyricist+'.txt', 'w') as f:
        for item in songs:
            f.write("%s\n" % item)
    data = np.genfromtxt(lyricist+'.txt',delimiter='\n',dtype=str)
    print(lyricist,len(data))